In [1]:
import torch
from torch.utils import data
import lightning as L
from lightning import Fabric
from lightning.pytorch import loggers as pl_loggers

In [2]:
%load_ext autoreload
%autoreload 2
import data_module
from model import ImageClassification, OodClassification, MyModel

In [3]:
# Load dataset
dataset = data_module.MnistOodRandomDataset()
n = len(dataset)
train_size = int(0.8 * n)
train_set, test_set = data.random_split(dataset, [train_size, n - train_size])
train_loader = data.DataLoader(test_set, batch_size=64, shuffle=True, num_workers=4)    # TODO: change to train_set
test_loader = data.DataLoader(test_set, batch_size=64, shuffle=False, num_workers=4)
print(n)

120000


In [4]:
# Create model
image_model = ImageClassification(num_classes=10)
ood_model = OodClassification()
model = MyModel(image_model, ood_model)

In [5]:
# Train model
# tb_logger = pl_loggers.TensorBoardLogger('/')
# csv_logger = pl_loggers.CSVLogger('/')
trainer = L.Trainer(max_epochs=30)
trainer.fit(model, train_loader, train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/wayne/anaconda3/envs/cs5340/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/t

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/wayne/anaconda3/envs/cs5340/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


{'acc': tensor(0.0400, device='cuda:0'), 'f1': tensor(0.0308, device='cuda:0')}
{'acc': tensor(0.5000, device='cuda:0'), 'f1': tensor(0., device='cuda:0')}


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

{'acc': tensor(0.9712, device='cuda:0'), 'f1': tensor(0.9714, device='cuda:0')}
{'acc': tensor(0.9973, device='cuda:0'), 'f1': tensor(0.9973, device='cuda:0')}


Validation: |          | 0/? [00:00<?, ?it/s]

{'acc': tensor(0.9798, device='cuda:0'), 'f1': tensor(0.9799, device='cuda:0')}
{'acc': tensor(0.9986, device='cuda:0'), 'f1': tensor(0.9986, device='cuda:0')}


/home/wayne/anaconda3/envs/cs5340/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
